In [1]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import cmath
import random
import numpy as np
from scipy.optimize import minimize
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.extensions import UnitaryGate


In [2]:
def convert_unitary_to_gate(theta):
    mat = [
        [cmath.cos(theta), cmath.sin(theta)],
        [cmath.sin(theta), -cmath.cos(theta)]
    ]
    return UnitaryGate(mat)

In [3]:
def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    circ.cz(qubits[0], qubits[1])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[1][iz], qubits[iz])

    circ.cz(qubits[1], qubits[2])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[2][iz], qubits[iz])

circ = QuantumCircuit(3)
apply_fixed_ansatz([0, 1, 2], [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ])
circ.draw()

┌───────┐      ┌───────┐      ┌───────┐
q_0: ┤ Ry(1) ├─■──■─┤ Ry(1) ├────■─┤ Ry(1) ├
     ├───────┤ │  │ ├───────┤    │ ├───────┤
q_1: ┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼─┤ Ry(1) ├
     ├───────┤    │ ├───────┤ │  │ ├───────┤
q_2: ┤ Ry(1) ├────■─┤ Ry(1) ├─■──■─┤ Ry(1) ├
     └───────┘      └───────┘      └───────┘

In [4]:
# Creates the Hadamard test

def had_test(gate_type, qubits, auxiliary_index, parameters):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters)

    for first in range (0, len(gate_type)):
        for ie in range (0, len(gate_type[0])):
            if (gate_type[first][ie] == 1):
                circ.cz(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 2):
                # print(2)
                circ.cx(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 3):
                circ.cy(auxiliary_index, qubits[ie])

    
    circ.h(auxiliary_index)
    
circ = QuantumCircuit(4)
had_test([ [0, 0, 0], [0, 0, 1] ], [1, 2, 3], 0, [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ])
circ.draw()

┌───┐                                   ┌───┐
q_0: ──┤ H ├─────────────────────────────────■─┤ H ├
     ┌─┴───┴─┐      ┌───────┐      ┌───────┐ │ └───┘
q_1: ┤ Ry(1) ├─■──■─┤ Ry(1) ├────■─┤ Ry(1) ├─┼──────
     ├───────┤ │  │ ├───────┤    │ ├───────┤ │      
q_2: ┤ Ry(1) ├─■──┼─┤ Ry(1) ├─■──┼─┤ Ry(1) ├─┼──────
     ├───────┤    │ ├───────┤ │  │ ├───────┤ │      
q_3: ┤ Ry(1) ├────■─┤ Ry(1) ├─■──■─┤ Ry(1) ├─■──────
     └───────┘      └───────┘      └───────┘

In [5]:
# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[1], 4)
    circ.cz(qubits[0], 4)
    circ.ccx(auxiliary, qubits[1], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[2], 4)
    circ.cz(qubits[1], 4)
    circ.ccx(auxiliary, qubits[2], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1, 2, 3], [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ], 0, q_reg)
circ.draw()

»
q0_0: ────■────────■────────■──────■───────■────■───────■──────■────────■────»
      ┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐    │    »
q0_1: ┤ Ry(1) ├────┼────────┼──────┼───■───┼────■───────■──┤ Ry(1) ├────┼────»
      └───────┘┌───┴───┐    │      │   │   │    │       │  └───────┘┌───┴───┐»
q0_2: ─────────┤ Ry(1) ├────┼──────■───┼───■────┼───────┼───────────┤ Ry(1) ├»
               └───────┘┌───┴───┐  │   │   │    │       │           └───────┘»
q0_3: ──────────────────┤ Ry(1) ├──┼───┼───┼────┼───■───┼────────────────────»
                        └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  »
q0_4: ───────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├──────────────────»
                                 └───┘   └───┘└───┘   └───┘                  »
«                                                                    
«q0_0: ────■──────■───────■────■───────■──────■────────■────────■────
«          │      │       │    │       │  ┌───┴───┐    │        │    
«q0_1: ────┼──────┼───────┼────■───────■──┤ Ry(1) ├────┼────────┼────
«          │      │       │    │       │  └───────┘┌───┴───┐    │    
«q0_2: ────┼──────┼───■───┼────┼───────┼───────────┤ Ry(1) ├────┼────
«      ┌───┴───┐  │   │   │    │       │           └───────┘┌───┴───┐
«q0_3: ┤ Ry(1) ├──■───┼───■────┼───■───┼────────────────────┤ Ry(1) ├
«      └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  └───────┘
«q0_4: ─────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├───────────────────────────
«               └───┘   └───┘└───┘   └───┘

In [6]:
# CHANGE THIS FOR DIFFERENT b!!!!!, THE theta controls the unitary
u = convert_unitary_to_gate(np.pi/3 * 4)
def control_b(auxiliary, qubits):
    # u_circuit = QuantumCircuit(1, name='u')
    # u_circuit.append(u)

    # bs_gate = u_circuit.to_gate()

    # print(bs_gate)
    cb = u.control(1)

    # circ.append(cb, [auxiliary, 1])

    for ia in qubits:
        circ.append(cb, [auxiliary, ia])
        # circ.ch(auxiliary, ia)

circ = QuantumCircuit(4)
control_b(0, [1, 2, 3])
circ.draw()

/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


q_0: ─────■──────────■──────────■─────
     ┌────┴────┐     │          │     
q_1: ┤ Unitary ├─────┼──────────┼─────
     └─────────┘┌────┴────┐     │     
q_2: ───────────┤ Unitary ├─────┼─────
                └─────────┘┌────┴────┐
q_3: ──────────────────────┤ Unitary ├
                           └─────────┘

In [7]:
# Create the controlled Hadamard test, for calculating <psi|psi>

def special_had_test(gate_type, qubits, auxiliary_index, parameters, reg):

    circ.h(auxiliary_index)

    control_fixed_ansatz(qubits, parameters, auxiliary_index, reg)

    for ty in range (0, len(gate_type)):
        if (gate_type[ty] == 1):
            circ.cz(auxiliary_index, qubits[ty])
        elif (gate_type[ty] == 2):
            circ.cx(auxiliary_index, qubits[ty])
        elif (gate_type[ty] == 3):
            circ.cy(auxiliary_index, qubits[ty])


    control_b(auxiliary_index, qubits)
    
    circ.h(auxiliary_index)

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
special_had_test([ [0, 2, 0], [3, 0, 1] ], [1, 2, 3], 0, [ [1, 1, 1], [1, 1, 1], [1, 1, 1] ], q_reg)
circ.draw()

┌───┐                                                              »
q13_0: ┤ H ├────■────────■────────■──────■───────■────■───────■──────■────»
       └───┘┌───┴───┐    │        │      │       │    │       │  ┌───┴───┐»
q13_1: ─────┤ Ry(1) ├────┼────────┼──────┼───■───┼────■───────■──┤ Ry(1) ├»
            └───────┘┌───┴───┐    │      │   │   │    │       │  └───────┘»
q13_2: ──────────────┤ Ry(1) ├────┼──────■───┼───■────┼───────┼───────────»
                     └───────┘┌───┴───┐  │   │   │    │       │           »
q13_3: ───────────────────────┤ Ry(1) ├──┼───┼───┼────┼───■───┼───────────»
                              └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐         »
q13_4: ────────────────────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├─────────»
                                       └───┘   └───┘└───┘   └───┘         »
«                                                                              »
«q13_0: ────■────────■──────■───────■────■───────■──────■────────■────────■────»
«           │        │      │       │    │       │  ┌───┴───┐    │        │    »
«q13_1: ────┼────────┼──────┼───────┼────■───────■──┤ Ry(1) ├────┼────────┼────»
«       ┌───┴───┐    │      │       │    │       │  └───────┘┌───┴───┐    │    »
«q13_2: ┤ Ry(1) ├────┼──────┼───■───┼────┼───────┼───────────┤ Ry(1) ├────┼────»
«       └───────┘┌───┴───┐  │   │   │    │       │           └───────┘┌───┴───┐»
«q13_3: ─────────┤ Ry(1) ├──■───┼───■────┼───■───┼────────────────────┤ Ry(1) ├»
«                └───────┘┌─┴─┐ │ ┌─┴─┐┌─┴─┐ │ ┌─┴─┐                  └───────┘»
«q13_4: ──────────────────┤ X ├─■─┤ X ├┤ X ├─■─┤ X ├───────────────────────────»
«                         └───┘   └───┘└───┘   └───┘                           »
«                                        ┌───┐
«q13_0: ─────■──────────■──────────■─────┤ H ├
«       ┌────┴────┐     │          │     └───┘
«q13_1: ┤ Unitary ├─────┼──────────┼──────────
«       └─────────┘┌────┴────┐     │          
«q13_2: ───────────┤ Unitary ├─────┼──────────
«                  └─────────┘┌────┴────┐     
«q13_3: ──────────────────────┤ Unitary ├─────
«                             └─────────┘     
«q13_4: ──────────────────────────────────────
«

## Simulation

In [8]:
coefficient_set = [0.5, 1.5]
gate_set = [ [2, 0, 0], [0, 0, 0] ]
# coefficient_set = [0.55, 0.45]
# gate_set = [ [0], [1] ]

# Implements the entire cost function on the quantum circuit

def calculate_cost_function(parameters):
    
    global opt

    overall_sum_1 = 0
    
    parameters = [parameters[0:3], parameters[3:6], parameters[6:9] ]

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            global circ

            qctl = QuantumRegister(5)
            qc = ClassicalRegister(5)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j] ], [1, 2, 3], 0, parameters)

            circ.save_statevector()
            t_circ = transpile(circ, backend)
            # qobj = assemble(t_circ)
            job = backend.run(t_circ)

            result = job.result()
            outputstate = np.real(result.get_statevector(circ, decimals=100))
            o = outputstate

            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = o[l]**2
                    m_sum+=n

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            multiply = coefficient_set[i]*coefficient_set[j]
            mult = 1

            for extra in range(0, 2):

                qctl = QuantumRegister(5)
                qc = ClassicalRegister(5)
                circ = QuantumCircuit(qctl, qc)

                backend = Aer.get_backend('aer_simulator')

                if (extra == 0):
                    special_had_test(gate_set[i], [1, 2, 3], 0, parameters, qctl)
                if (extra == 1):
                    special_had_test(gate_set[j], [1, 2, 3], 0, parameters, qctl)

                circ.save_statevector()    
                t_circ = transpile(circ, backend)
                # qobj = assemble(t_circ)
                job = backend.run(t_circ)

                result = job.result()
                outputstate = np.real(result.get_statevector(circ, decimals=100))
                o = outputstate

                m_sum = 0
                for l in range (0, len(o)):
                    if (l%2 == 1):
                        n = o[l]**2
                        m_sum+=n
                mult = mult*(1-(2*m_sum))

            overall_sum_2+=multiply*mult
            
    # print(1-float(overall_sum_2/overall_sum_1))

    return 1-float(overall_sum_2/overall_sum_1)

In [9]:
from qiskit import IBMQ
# load IBM account
IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
IBMQ.load_account()

provider = IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-education', group='harvard', project='qse-210')

/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_56635/2447371092.py:3: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_56635/2447371092.py:3: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qisk

In [ ]:
backend=provider.get_backend("ibm_perth")
# backend=provider.get_backend("ibmq_quito")

In [14]:


out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9] ]



 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 6.774292238276303e-09
       x: [ 3.142e+00  1.542e+00  1.948e+00  1.412e+00  2.292e+00
            1.948e+00  3.300e+00  8.210e-01  1.571e+00]
    nfev: 144
   maxcv: 0.0


In [60]:
circ = QuantumCircuit(3, 3)
apply_fixed_ansatz([0, 1, 2], out_f)
circ.save_statevector()

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
print(circ)

job = backend.run(t_circ, shots=8192)

result = job.result()
print(result.get_counts())
o = result.get_statevector(circ, decimals=10)
print(o)
print(o[0]/o[4])

x_sol_not_normalized = np.array([o[0]/o[4], 1])
x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

# a1 = coefficient_set[1]*np.array([ [1,0], [0,-1] ])
# a2 = coefficient_set[0]*np.array([ [1,0], [0,1]])
# a1 = coefficient_set[1]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,-1,0,0,0], [0,0,0,0,0,-1,0,0], [0,0,0,0,0,0,-1,0], [0,0,0,0,0,0,0,-1] ])
# # a2 = coefficient_set[0]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1] ])
# a2i = coefficient_set[1]*np.array([ [1,0,0,0,0,0,0,0], [0,1,0,0,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,0,0,1,0], [0,0,0,0,0,0,0,1] ])

# a2x = coefficient_set[0]*np.array([ [0,1,0,0,0,0,0,0], [1,0,0,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,1,0] ])

# a3 = np.add(a2x, a2i)

a = np.array([[1.5, 0.5], [0.5, 1.5]])

b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

# b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

# print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o))))))

print((b.dot(a.dot(x_normalized)/(np.linalg.norm(a.dot(x_normalized))))))



     ┌────────────┐      ┌────────────┐       ┌────────────┐ statevector 
q_0: ┤ Ry(3.1415) ├─■──■─┤ Ry(1.4122) ├────■──┤ Ry(3.3003) ├──────░──────
     ├────────────┤ │  │ ├────────────┤    │ ┌┴────────────┤      ░      
q_1: ┤ Ry(1.5425) ├─■──┼─┤ Ry(2.2922) ├─■──┼─┤ Ry(0.82103) ├──────░──────
     ├────────────┤    │ ├────────────┤ │  │ └┬────────────┤      ░      
q_2: ┤ Ry(1.9476) ├────■─┤ Ry(1.9475) ├─■──■──┤ Ry(1.5706) ├──────░──────
     └────────────┘      └────────────┘       └────────────┘      ░      
c: 3/════════════════════════════════════════════════════════════════════
                                                                         
{'000': 8192}
Statevector([-0.35354344+0.j, -0.35362764+0.j, -0.35354826+0.j,
             -0.35359221+0.j, -0.35353668+0.j, -0.35350769+0.j,
             -0.35351001+0.j, -0.35356119+0.j],
            dims=(2, 2, 2))
(1.0000191055139926-0j)
(0.9999999999885933+0j)


In [104]:
def extract_counts(dict):
    total_counts = 0
    relevant_count0 = 0
    relevant_count1 = 0
    for key in dict:
        if key[0] == '1':
            if key[-1] == '1':
                relevant_count1 += int(dict[key])
            if key[-1] == "0":
                relevant_count0 += int(dict[key])
        total_counts += int(dict[key])
    return (relevant_count0/total_counts, relevant_count1/total_counts)

def extract_solution(probs):
    return (np.sqrt(probs[0]), np.sqrt(probs[1]))

def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

In [100]:
def paper_no2_sim():
    global circ

    # coefficient_set = [0.5, 1.5]
    # gate_set = [ [2, 0, 0], [0, 0, 0] ]
    backend = Aer.get_backend('aer_simulator')

    out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
    print(out)

    out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9] ]

    circ = QuantumCircuit(3, 3)
    apply_fixed_ansatz([0, 1, 2], out_f)
    circ.save_statevector()

    t_circ = transpile(circ, backend)
    print(circ)

    job = backend.run(t_circ, shots=8192)

    result = job.result()
    print(result.get_counts())
    o = result.get_statevector(circ, decimals=10)
    print(o)
    print(o[0], o[4], o[0]/o[4])

    x_sol_not_normalized = np.array([o[0]/o[4], 1])
    x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

    a = np.array([[1.5, 0.5], [0.5, 1.5]])

    b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

    classical_sol = (0.354, 0.354)

    f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
    print(x_sol_not_normalized, x_normalized)
    print("fidelity: ", f2)
    print("params: ", out_f)

paper_no2_sim()



 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 3.319385693645316e-05
       x: [ 2.816e-02  3.661e+00  2.361e+00 -2.037e-02  2.625e+00
            3.432e+00  1.561e+00  1.570e+00  2.062e+00]
    nfev: 200
   maxcv: 0.0
     ┌──────────────┐      ┌───────────────┐      ┌────────────┐ statevector 
q_0: ┤ Ry(0.028165) ├─■──■─┤ Ry(-0.020371) ├────■─┤ Ry(1.5606) ├──────░──────
     └┬────────────┬┘ │  │ └─┬────────────┬┘    │ ├────────────┤      ░      
q_1: ─┤ Ry(3.6609) ├──■──┼───┤ Ry(2.6255) ├──■──┼─┤ Ry(1.5704) ├──────░──────
      ├────────────┤     │   ├────────────┤  │  │ ├────────────┤      ░      
q_2: ─┤ Ry(2.3614) ├─────■───┤ Ry(3.4319) ├──■──■─┤ Ry(2.0622) ├──────░──────
      └────────────┘         └────────────┘       └────────────┘      ░      
c: 3/════════════════════════════════════════════════════════════════════════
                                                                             
{'000': 8192}
Statev

In [110]:
def paper_no2_sim_3pi4():
    global circ

    coefficient_set = [0.5, 1.5]
    gate_set = [ [2, 0, 0], [0, 0, 0] ]
    backend = Aer.get_backend('aer_simulator')

    out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
    print(out)

    out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9] ]

    circ = QuantumCircuit(3, 3)
    apply_fixed_ansatz([0, 1, 2], out_f)
    circ.save_statevector()

    t_circ = transpile(circ, backend)
    print(circ)

    job = backend.run(t_circ, shots=8192)

    result = job.result()
    print(result.get_counts())
    o = result.get_statevector(circ, decimals=10)
    print(o)
    print(o[0], o[4], o[0]/o[4])

    x_sol_not_normalized = np.array([o[0]/o[4], 1])
    x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

    a = np.array([[1.5, 0.5], [0.5, 1.5]])

    b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

    classical_sol = (-0.158, -0.525)

    f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
    print(x_sol_not_normalized, x_normalized)
    print("fidelity: ", f2)
    print("params: ", out_f)

paper_no2_sim_3pi4()


 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: 5.791522805886906e-06
       x: [ 1.212e-02  4.770e-01  3.580e+00  2.261e+00  3.753e+00
            2.704e+00  3.048e-01  4.150e+00  2.089e+00]
    nfev: 200
   maxcv: 0.0
     ┌──────────────┐      ┌────────────┐      ┌─────────────┐ statevector 
q_0: ┤ Ry(0.012118) ├─■──■─┤ Ry(2.2606) ├────■─┤ Ry(0.30479) ├──────░──────
     ├─────────────┬┘ │  │ ├────────────┤    │ └┬────────────┤      ░      
q_1: ┤ Ry(0.47698) ├──■──┼─┤ Ry(3.7527) ├─■──┼──┤ Ry(4.1505) ├──────░──────
     └─┬──────────┬┘     │ ├────────────┤ │  │  ├────────────┤      ░      
q_2: ──┤ Ry(3.58) ├──────■─┤ Ry(2.7036) ├─■──■──┤ Ry(2.0894) ├──────░──────
       └──────────┘        └────────────┘       └────────────┘      ░      
c: 3/══════════════════════════════════════════════════════════════════════
                                                                           
{'000': 8192}
Statevector([0.07183644+

In [118]:
def paper_no2_hardware():
    global circ

    coefficient_set = [0.5, 1.5]
    gate_set = [ [2, 0, 0], [0, 0, 0] ]
    backend=provider.get_backend("ibmq_quito")
    
    out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
    print(out)

    out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9] ]

    circ = QuantumCircuit(3, 3)
    apply_fixed_ansatz([0, 1, 2], out_f)
    circ.measure_all()

    t_circ = transpile(circ, backend)
    print(circ)

    job = backend.run(t_circ, shots=8192, job_tags=["quito-1.5-.5-b=maxmixed-t=pi/2"])

    result = job.result()
    print(result.get_counts())
    o = result.get_counts(circ)
    print(o)
    print(o["000 000"],o["100 000"], o["000 000"]/o["100 000"])

    x_sol_not_normalized = np.array([o["000 000"]/o["100 000"], 1])
    x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

    a = np.array([[1.5, 0.5], [0.5, 1.5]])

    b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

    classical_sol = (0.354, 0.354)

    f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
    print(x_sol_not_normalized, x_normalized)
    print("fidelity: ", f2)
    print("params: ", out_f)

paper_no2_hardware()


Traceback (most recent call last):
  File "/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/scipy/optimize/_cobyla_py.py", line 258, in calcfc
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
    f = fun(np.copy(x), *args)
        ^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_52977/1433544192.py", line 70, in calculate_cost_function
  File "/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/qiskit/compiler/transpiler.py", line 381, in transpile
    _serial_transpile_circuit(
  File "/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/qiskit/compiler/transpiler.py", line 474, in _serial_transpile_circuit
    result = pass_manager.run(circuit, callback=callback, output_name=output_name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^

: 

: 

In [36]:

o = {'000 000': 1043, '010 000': 960, '011 000': 940, '100 000': 1182, '101 000': 1159, '110 000': 910, '111 000': 829, '001 000': 1169}
print(o)
print(o["000 000"], o["100 000"], o["000 000"]/o["100 000"])

x_sol_not_normalized = np.array([o["000 000"]/o["100 000"], 1])
x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

a = np.array([[1.5, 0.5], [0.5, 1.5]])

b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

classical_sol = (0.354, 0.354)

f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
print(x_sol_not_normalized, x_normalized)
print("fidelity: ", f2)
print("params: ", out_f)

{'000 000': 1043, '010 000': 960, '011 000': 940, '100 000': 1182, '101 000': 1159, '110 000': 910, '111 000': 829, '001 000': 1169}
1043 1182 0.8824027072758037
[0.88240271 1.        ] [0.66164262 0.74981934]
fidelity:  0.9980543234763987
params:  [array([3.14151595, 1.54247113, 1.94763865]), array([1.41222224, 2.29223399, 1.94752469]), array([3.30027225, 0.82103079, 1.57059807])]


In [ ]:
c = extract_counts(result.get_counts())
s = extract_solution(c)
print(c, s)

(0.529052734375, 0.4615478515625) (0.7273601132692114, 0.679373131322177)


In [10]:
def paper_no2_hardware3pi4():
    global circ

    coefficient_set = [0.5, 1.5]
    gate_set = [ [2, 0, 0], [0, 0, 0] ]
    backend=provider.get_backend("ibmq_quito")
    
    out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
    print(out)

    out_f = [out['x'][0:3], out['x'][3:6], out['x'][6:9] ]

    circ = QuantumCircuit(3, 3)
    apply_fixed_ansatz([0, 1, 2], out_f)
    circ.measure_all()

    t_circ = transpile(circ, backend)
    print(circ)

    job = backend.run(t_circ, shots=8192, job_tags=["quito-1.5-.5-t=3pi/4"])

    result = job.result()
    print(result.get_counts())
    o = result.get_counts(circ)
    print(o)
    print(o["000 000"],o["100 000"], o["000 000"]/o["100 000"])

    x_sol_not_normalized = np.array([o["000 000"]/o["100 000"], 1])
    x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

    a = np.array([[1.5, 0.5], [0.5, 1.5]])

    b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

    classical_sol = (0.354, 0.354)

    f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
    print(x_sol_not_normalized, x_normalized)
    print("fidelity: ", f2)
    print("params: ", out_f)

paper_no2_hardware3pi4()


/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [ ]:
quito1 = {'000': 348, '001': 255, '010': 222, '011': 275, '100': 2117, '101': 1530, '110': 2048, '111': 1397}
quito2 =  {'000': 350, '001': 247, '010': 189, '011': 258, '100': 2085, '101': 1571, '110': 2067, '111': 1425}
quito3 = {'000': 365, '001': 259, '010': 242, '011': 260, '100': 2164, '101': 1531, '110': 1983, '111': 1388}
quito4 = {'000': 352, '001': 243, '010': 225, '011': 239, '100': 2139, '101': 1603, '110': 2054, '111': 1337}
quito5 = {'000': 315, '001': 282, '010': 209, '011': 267, '100': 2173, '101': 1527, '110': 2018, '111': 1401}
q = [quito1, quito2, quito3, quito4, quito5]

In [ ]:
perth1 = {'000': 196, '001': 193, '010': 195, '011': 163, '100': 1937, '101': 1305, '110': 2509, '111': 1694}
perth2 = {'000': 197, '001': 162, '010': 216, '011': 158, '100': 1876, '101': 1339, '110': 2582, '111': 1662}
perth3 =  {'000': 184, '001': 188, '010': 207, '011': 170, '100': 1838, '101': 1289, '110': 2604, '111': 1712}
perth4 = {'000': 155, '001': 200, '010': 211, '011': 168, '100': 1816, '101': 1292, '110': 2597, '111': 1753}
perth5 = {'000': 189, '001': 191, '010': 204, '011': 161, '100': 1964, '101': 1273, '110': 2555, '111': 1655}
p = [perth1, perth2, perth3, perth4, perth5]

In [ ]:
p_avg_f = 0
q_avg_f = 0

for qrun in q:
    probs = extract_counts(qrun)